In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 1
Write the functions to achieve 

(1) Arnoldi iteration and Lanczos iteration; 

(2) QR iteration for Hessenberg matrix to find the eigenvalue decomposition.

 Test these algorithms for a few matrix.

In [ ]:
def Arnoldi_iter(A, x_0):
    '''Arnoldi Iteration'''
    n = A.shape[0] # 初始化设置
    Q = np.zeros([n, n]); Q[:, 0] = x_0 / np.linalg.norm(x_0, ord=2)
    U, H = np.zeros([n, n]), np.zeros([n, n])

    # 循环主体
    for k in range(n):
        U[:, k] = A @ Q[:, k]
        for j in range(k+1):
            H[j, k] = Q[:, j].T @ U[:, k]
            U[:, k] = U[:, k] - H[j, k] * Q[:, j]
        try:
            H[k+1, k] = np.linalg.norm(U[:, k], ord=2)
            if H[k+1, k]==0:
                break
            Q[:, k+1] = U[:, k] / H[k+1, k]
        except:
            pass

    return Q, U, H

# TEST
A = np.random.random([4, 4])
x_0 = np.ones(4)
Q, U ,H = Arnoldi_iter(A, x_0)
A @ Q - Q @ H

In [ ]:
def Lanczos_iter(A, x_0):
    '''Lanczos Iteration'''
    # 初始化设置
    n = A.shape[0]
    Q = np.zeros([n, n + 1]); Q[:, 1] = x_0 / np.linalg.norm(x_0, ord=2)
    beta = np.zeros(n + 1); alpha = np.zeros(n + 1)
    U = np.zeros([n, n+1])
    H = np.zeros([n, n])

    for k in range(1, n+1):
        U[:, k] = A @ Q[:, k]
        alpha[k] = Q[:, k].T @ U[:, k]
        U[:, k] = U[:, k] - beta[k-1] * Q[:, k-1] - alpha[k] * Q[:, k]
        beta[k] = np.linalg.norm(U[:, k], ord=2)
        if beta[k]==0:
            break
        try:
            Q[:, k+1] = U[:, k] / beta[k]
        except:
            pass
    for i in range(n):
        H[i, i] = alpha[i+1]

    for i in range(n-1):
        H[i, i+1] = beta[i+1]
        H[i+1, i] = beta[i+1]

    Q = Q[:, 1:n+1]
    return Q, U, H

# TEST
A = np.random.random([4, 4])
x_0 = np.ones(4)
Q, U ,H = Arnoldi_iter(A, x_0)
A @ Q - Q @ H

In [ ]:
def QRSplit(A):
    '''QR iteration'''
    n = A.shape[0]; R = A.copy()
    for i in range(0, n-1):
        B = R.copy()
        if i != 0:
            B = B[i:,i:]
        x = B[:,0]
        y = np.zeros(n-i); y[0] = np.linalg.norm(x)
        w = x - y; w = w / np.linalg.norm(w)
        H = np.eye(n-i) - 2*np.dot(w.reshape(n-i, 1), w.reshape(1, n-i))
        if i == 0: 
            Q = H.copy(); R = H @ R
        else:
            D = np.c_[np.eye(i), np.zeros((i, n-i))]
            H = np.c_[np.zeros((n-i, i)), H]
            H = np.r_[D, H]
            Q = H @ Q
            R = H @ R
    Q = Q.T
    return Q, R

H = np.random.random([4, 4])
for i in range(4):
    for j in range(4):
        if i>j+1:
            H[i, j] = 0
print(H)
for i in range(100):
    Q, R = QRSplit(H)
    H = R @ Q
print(H)

### 2
For a second order differential equation $\left(D(x) u^{\prime}(x)\right)^{\prime}=b(x)$, a central difference method to solve the equation is $\frac{D\left(x_{i+\frac{1}{2}}\right)\left(u\left(x_{i+1}\right)-u\left(x_i\right)\right)-D\left(x_{i-\frac{1}{2}}\right)\left(u\left(x_i\right)-u\left(x_{i-1}\right)\right)}{h^2}=b\left(x_i\right)$, where $x_i$ are equally spaced nodes in interval $[a, b](i=0,1,2, \ldots, N), x_{i+\frac{1}{2}}$ are the midpoint of $x_i$ and $x_{i+1}$, and $h=$ $\frac{b-a}{N}$. For the following cases,

1) $\quad D(x)=1,[a, b]=[-1,1], N=100$;
   
2) $\quad D(x)=e^x,[a, b]=[-5,5], N=100$
   

Construct the coefficient matrix of the discretized linear system, then calculate the eigenvalues, eigenvectors, and condition number of the system. Compare the results for the two cases. For the second case, if you divide each row of the matrix by the corresponding diagonal element (and the right hand side term of the linear equation), how would it change the condition number?

第一种情况已经在作业纸上写明；下面只讨论第二种情况

In [14]:
D_2: lambda x: np.exp(x)

a_2 = -5
b_2 = 5
N = 100

x_2 = np.linspace(a_2, b_2, N+1, endpoint=True)
d_2 = (b_2 - a_2) / N

D2 = np.zeros([N+1, N+1])
for i in range(N+1):
    D2[i, i] = - D_2(x_2[i]+d_2/2) - D_2(x_2[i]-d_2/2)


for i in range(N):
    D2[i, i+1] = D_2(x_2[i]+d_2/2)
    D2[i+1, i] = D_2(x_2[i]+d_2/2)

print(D2)

[[-1.34927424e-02  7.08340893e-03  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 7.08340893e-03 -1.49117865e-02  7.82837755e-03 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  7.82837755e-03 -1.64800728e-02 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ... -2.43324674e+02
   1.27740390e+02  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  1.27740390e+02
  -2.68915354e+02  1.41174964e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   1.41174964e+02 -2.97197428e+02]]


可见 D2 和 第一问中的情况相同，都是三对角矩阵

In [15]:
# 不处理
Lambda2, P2 = np.linalg.eig(D2)
condition2 = np.linalg.norm(D2) * np.linalg.norm(np.linalg.inv(D2))
print(condition2)
# divide each row of the matrix by the corresponding diagonal element
D2 = D2 / np.diagonal(D2)
condition2 = np.linalg.norm(D2) * np.linalg.norm(np.linalg.inv(D2))
print(condition2)

3985599.4087922075
13597.619254051388


可见经过这样的调整之后条件数大大减小

### 3
Write the functions to achieve Newton’s method and Broyden’s method. Test these methods for a few equation systems.

In [ ]:
""" 牛顿法 """
f1 = lambda x: 2 * x[0]**2 + x[1]**2 - 1
f2 = lambda x: 5 * x[0] * x[1]
J = lambda x: [[4 * x[0], 2 * x[1]], [5 * x[1], 5 * x[0]]]

x = [0.1, 0.5]
for i in range(10):
    x = x - np.linalg.inv(J(x)) @ [f1(x), f2(x)]
print(x)

In [ ]:
""" Broyden """
x = np.zeros([11, 2])
x[0] = [0.1, 0.9]
B = J(x[0])
for i in range(1, 10):
    s = - np.linalg.inv(B) @ [f1(x[i]), f2(x[i])]
    x[i+1] = x[i] + s
    y = np.array([f1(x[i+1]) - f1(x[i]), f2(x[i+1]) - f2(x[i])])
    B = B + ((y - B @ s) @ s.T) / (s.T @ s)
print(x[-1])

可见都能达到真实解